In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np


In [20]:
#constant
TEST_EMBEDDING_FILE = "test_embeddings.pt"
TRAIN_EMBEDDING_FILE = "train_embeddings.pt"
VAL_EMBEDDING_FILE = "val_embeddings.pt"

GNN_TEST_EMBEDDING = f"../GNN/embeddings/{TEST_EMBEDDING_FILE}"
GNN_TRAIN_EMBEDDING = f"../GNN/embeddings/{TRAIN_EMBEDDING_FILE}"
GNN_VAL_EMBEDDING = f"../GNN/embeddings/{VAL_EMBEDDING_FILE}"
TOXBERT_TEST_EMBEDDING = f"../ToxBERT/embedding/{TEST_EMBEDDING_FILE}"
TOXBERT_TRAIN_EMBEDDING = f"../ToxBERT/embedding/{TRAIN_EMBEDDING_FILE}"
TOXBERT_VAL_EMBEDDING = f"../ToxBERT/embedding/{VAL_EMBEDDING_FILE}"

In [33]:
def load_embedding_file(path):
    """
    Hàm load file embedding từ đường dẫn `path`.
    Tự động nhận dạng nhiều kiểu dữ liệu khác nhau:
    - dict có key 'embeddings' hoặc 'emb', 'vectors', 'features'
    - trực tiếp là Tensor
    - list/tuple (chuyển sang Tensor)
    """
    # ---- Bỏ chặn numpy pickle cổ ----
    try:
        torch.serialization.add_safe_globals([np.core.multiarray._reconstruct])
    except Exception:
        pass  # không sao

    # Load thử với weights_only=False ----
    try:
        data = torch.load(path, weights_only=False)
    except Exception as e:
        raise RuntimeError(f"LỖI load file embedding: {e}")
    
    # data = torch.load(path, map_location='cpu')

    #Chuẩn hóa về tensor ----
    # Nếu là tensor trực tiếp
    if isinstance(data, torch.Tensor):
        return {'embeddings': data}
    

    # Nếu là numpy
    if isinstance(data, np.ndarray):
        return {'embeddings': torch.tensor(data)}
    
    # Trường hợp 1: File lưu dạng dict
    if isinstance(data, dict):

        # TH1.1: Dict có key phổ biến 'embeddings'
        if 'embeddings' in data:
            return {
                'embeddings': torch.as_tensor(data['embeddings']),
                'ids': data.get('ids', None)  # có thể không có
            }

        # TH1.2: Một số file embedding dùng key khác
        for key in ['emb', 'vectors', 'features']:
            if key in data:
                return {
                    'embeddings': torch.as_tensor(data[key]),
                    'ids': data.get('ids', None)
                }

        # TH1.3: Dict không rõ cấu trúc → thử convert cả dict sang tensor
        try:
            return {'embeddings': torch.as_tensor(data)}
        except Exception:
            raise ValueError(
                f"Lỗi: Dict trong file {path} có cấu trúc không hỗ trợ để chuyển sang Tensor"
            )

    

    # Trường hợp 3: File là list hoặc tuple → convert sang Tensor
    elif isinstance(data, (list, tuple)):
        try:
            return {'embeddings': torch.as_tensor(data)}
        except Exception:
            raise ValueError(
                f"Lỗi: Không thể chuyển list/tuple trong file {path} sang Tensor"
            )

    # Trường hợp không thuộc loại nào
    else:
        raise ValueError(
            f"Lỗi: Định dạng dữ liệu trong file {path} không được hỗ trợ"
            f"Loại: {type(data)}"
        )


In [38]:
import os
import torch




def check_embedding_file(path):
    """Kiểm tra 1 file embedding và trả về True/False + in thông tin chi tiết."""

    print(f"KIỂM TRA FILE: {path} ======")

    if not os.path.exists(path):
        print("Lỗi: File không tồn tại.")
        return False

    try:
        result = load_embedding_file(path)
        emb = result["embeddings"]

        # --- Kiểm tra embedding có phải Tensor ---
        if not isinstance(emb, torch.Tensor):
            print("Lỗi: embeddings không phải Torch.Tensor.")
            return False

        # --- Kiểm tra số chiều ---
        if emb.ndim < 2:
            print(f"Lỗi: embeddings phải >= 2 chiều, hiện tại: {emb.ndim}")
            return False

        # --- In thông tin hợp lệ ---
        print("✅File hợp lệ!")
        print(f"Kích thước: {tuple(emb.shape)}___Kiểu dữ liệu: {emb.dtype}___Có IDs không? { 'Có' if result.get('ids') is not None else 'Không' }")
       
        return True

    except Exception as e:
        print(f"Lỗi khi load file: {e}")
        return False


def test_all_embeddings():
    """Test toàn bộ file ToxBERT + GNN"""

    toxbert_files = [
        TOXBERT_TEST_EMBEDDING,
        TOXBERT_TRAIN_EMBEDDING,
        TOXBERT_VAL_EMBEDDING,
    ]

    gnn_files = [
        GNN_TEST_EMBEDDING,
        GNN_TRAIN_EMBEDDING,
        GNN_VAL_EMBEDDING,
    ]

   
    print("KIỂM TRA TOXBERT BRANCH==================================")
  

    for f in toxbert_files:
        check_embedding_file(f)

    
    print("KIỂM TRA GNN BRANCH====================================")

    for f in gnn_files:
        check_embedding_file(f)


if __name__ == "__main__":
    test_all_embeddings()



KIỂM TRA TOXBERT BRANCH==================================
KIỂM TRA FILE: ../ToxBERT/embedding/test_embeddings.pt ======
✅File hợp lệ!
Kích thước: (3264, 768)___Kiểu dữ liệu: torch.float32___Có IDs không? Không
KIỂM TRA FILE: ../ToxBERT/embedding/train_embeddings.pt ======
✅File hợp lệ!
Kích thước: (15227, 768)___Kiểu dữ liệu: torch.float32___Có IDs không? Không
KIỂM TRA FILE: ../ToxBERT/embedding/val_embeddings.pt ======
✅File hợp lệ!
Kích thước: (3263, 768)___Kiểu dữ liệu: torch.float32___Có IDs không? Không
KIỂM TRA GNN BRANCH====================================
KIỂM TRA FILE: ../GNN/embeddings/test_embeddings.pt ======
✅File hợp lệ!
Kích thước: (802, 512)___Kiểu dữ liệu: torch.float32___Có IDs không? Không
KIỂM TRA FILE: ../GNN/embeddings/train_embeddings.pt ======
✅File hợp lệ!
Kích thước: (6411, 512)___Kiểu dữ liệu: torch.float32___Có IDs không? Không
KIỂM TRA FILE: ../GNN/embeddings/val_embeddings.pt ======
✅File hợp lệ!
Kích thước: (801, 512)___Kiểu dữ liệu: torch.float32___Có I